In [1]:
import tensorflow as tf

In [2]:
weights = tf.get_variable("weights", [5, 5, 3, 16],
                         initializer=tf.truncated_normal_initializer(stddev=0.1))
biases = tf.get_variable("biases", [16],
                        initializer=tf.constant_initializer(0.1))

卷积层

In [9]:
input = tf.placeholder(tf.float32,[1000,28,28,3])
#每个batch有1000个数据 28*28*3
conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding="SAME")
#步长参数第一维和最后一维一定是1
bias = tf.nn.bias_add(conv, biases)
actived_conv = tf.nn.relu(bias)
#注意bias不能简单相加

池化层

In [10]:
pool=tf.nn.max_pool(actived_conv, ksize=[1,3,3,1],
                   strides=[1,2,2,1],padding="SAME")
#ksize过滤器尺寸 strides步长 参数与conv2d类似第一维和最后一维一定是1

dropout层（一般只有全连接层有dropout）

In [11]:
train = True
if train:
    pool = tf.nn.dropout(pool, 0.5)

# TFRecord

TensorFlow提供统一的格式来存储数据（TFRecord）

In [ ]:
#例子 如何转化为TFRecord格式
#有很多WARNING
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

mnist = input_data.read_data_sets(
"/path/to/mnist/data",dtype=tf.uint8,one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

#输出TFRecord文件地址
filename = "/path/to/output.tfrecords"
writer = tf.python_io.TFRecordWriter(filename)
for index in range(num_examples):
    image_raw = image[indes].tostring()
    example = tf.train.Example(features = tf.train.Features(features={
    "pixels":_int64_feature(pixels),
    "label":_int64_feature(np.argmax(labels[index])),
    "image_raw":_bytes_feature(image_raw)}))
    
    writer.write(example.SerializeToString())
writer.close()


In [ ]:
#例子 读取TFRecord文件
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["/path/to/output.tfrecords"])
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
serialized_example,
features = {
    "image_raw":tf.FixedLenFeature([], tf.string),
    "pixels": tf.FixedLenFeature([], tf.int64),
    "label":tf.FixedLenFeature([], tf.int64)
})
images = tf.decode_raw(features["image_raw"],tf.uint8)
labels = tf.cast(features["label"],tf.int32)
pixels = tf.cast(features["pixels"],tf.int32)

sess = tf.Session()
coord = tf.train.Coordinator() #多线程处理
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(10):
    image, label, pixel = sess.run([images,labels,pixels])

# 图像处理

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
#读取图片
image_raw_data = tf.gfile.FastGFile("/path/to/picture","r").read()
with tf.Session() as sess:
    img_data = tf.image.decode_jpeg(image_raw_data)
#展示图片 img_data为三维数组
plt.imshow(img_data.eval())
plt.show()
#转化为实数
img_data = tf.image.convert_image_dtype(img_data, dtype=tf.float32)
#编码为jpeg
encoded_image = tf.image.encode_jpeg(img_data)
with tf.gfile.GFile("/path/to/output","wb") as f:
    f.write(encoded_image.eval())

图片大小调整

In [ ]:
#method 0：双线性 1：最近邻 2：双三次 3：面积法
tf.image.resize_images(img_data,300,300,method=0)
#获取图像大小
img_data.get_shape()

图像裁剪或填充

In [ ]:
tf.image.resize_image_with_crop_or_pad(img_data,1000,1000)

比例调整

In [ ]:
tf.image.central_crop(img_data,0.5)

图像翻转

In [ ]:
#上下翻转
tf.image.flip_up_down(img_data)
#左右翻转
tf.image.filp_left_right(img_data)
#对角线翻转
tf.image.transpose_image(img_data)
#一定概率翻转
tf.image.random_flip_up_down(img_data)
tf.image.random_flip_left_right(img_data)

色彩调节

In [ ]:
#调整亮度
tf.image.adjust_brightness(img_data,-0.5)
#在-max_delta和max_delta之间随机调整亮度
tf.image.random_brightness(img_data, max_delta)
#调整对比度
tf.image.adjust_contrast(img_data, -5)
#在low-up之间随机调整对比度
tf.image.random_contrast(img_data, low, up)
#调整色相
tf.image.adjust_hue(img_data, 0.5)
#max_delta取值0-0.5
tf.image.random_hue(image, max_delta)
#调整饱和度
tf.image.adjust_saturation(img_data, -5)
#在low-up之间随机调整饱和度
tf.image.random_saturation(img_data, low, up)

#净化
#将图片数字均值变为0方差变为1
tf.image.per_image_whitening(img_data)

处理标注框

In [ ]:
tf.image.draw_bounding_boxes(img, box)
#box=[y_min,x_min,y_max,x_max]都是相对坐标

截取图像

In [ ]:
#在标注框内随机截取
tf.image.sample_distorted_bounding_box(img_data,box)

# 循环神经网络

In [3]:
import numpy as np
x = [1, 2]
state = [0, 0]

w_cell_state = np.array([[0.1,0.2],[0.3,0.4]])
w_cell_input = np.array([0.5,0.6])
b_cell = np.array([0.1, -0.1])

#全连接层
w_output = np.array([[1.0],[2.0]])
b_output = 0.1

for i in range(len(x)):
    before = np.dot(state, w_cell_state)+x[i]*w_cell_input+b_cell
    state = np.tanh(before)
    
    output = np.dot(state, w_output)+b_output
    print("before",before)
    print("state",state)
    print("output",output)

before [0.6 0.5]
state [0.53704957 0.46211716]
output [1.56128388]
before [1.2923401  1.39225678]
state [0.85973818 0.88366641]
output [2.72707101]


LSTM

通过tf.nn.rnn_cell.BasicLSTMCell实现

In [ ]:
lstm_size = [2,3]
#定义LSTM单元
lstm = tf.nn.rnn_cell.LSTMCell(lstm_size) #原BasicLSTMCell
#dropout
dropout_lstm = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=0.5)
#定义深层循环网络
number_of_layers = 3
stacked_lstm = tf.nn.rnn_cell.MultiRNNCell([dropout_lstm]*number_of_layers)

实例：

PTB数据集

tf.models.nn.ptb.reader.ptb_iterator(train_data, 4, 5)

# TFlearn

高层封装TFlearn 

导入方法 tf.contrib.learn

In [ ]:
#logistic回归为例
logits, loss = tf.contrib.learn.models.logistic_regression(features, target)